# 행정동 별 사망사고 계층화하기

## 행정동 코드 가져오기

In [3]:
import geopandas as gpd
import pandas as pd
import os
import numpy as np
import requests
import folium
from folium.plugins import MarkerCluster
import re


## 천안시 데이터 추출하기

In [4]:
acc=pd.read_csv('./사망사고 위치 위도 경도 파악.csv',encoding='cp949')
acc
cheonahn=acc[acc["발생지시군구"].str.contains('천안시')]


In [5]:
# 경도 위도 추출
lenght=len(cheonahn)
loca=[]
for i in range(0,lenght):
    x=cheonahn.iloc[i]['경도']
    y=cheonahn.iloc[i]['위도']
    loca.append([x,y])

## 네이버 API 이용해 행정동,법정동 코드,지역이름 꺼내기

In [6]:
# 네이버 API key 가져오기
def getAPIkey():
    f=open('./secret_key.txt',"r")
    return f.read()

In [7]:
CLIENT_ID = 'vjgu0f53rm'
key=getAPIkey()
CLIENT_SECRET = key
amd_code=[]
legal_code=[]
amd_name=[]
legal_name=[]
local_name=[]
for i in loca:
	x,y=i
	coords_param=f'{x},{y}'
	headers = {'X-NCP-APIGW-API-KEY-ID': CLIENT_ID, 'X-NCP-APIGW-API-KEY': CLIENT_SECRET}
	url2=f"https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc?coords={coords_param}&orders={'admcode,legalcode'}&output={'json'}"
	response2 = requests.get(url2, headers=headers)
	response_data2=response2.json()
	amd_code.append(response_data2['results'][0]['code']['mappingId'])
	legal_code.append(response_data2['results'][1]['code']['id'])
	data1=response_data2['results'][0]['region']['area2']['name']
	data2=response_data2['results'][0]['region']['area3']['name']
	data1=data1.replace(' ','')
	amd_name.append(data1+" "+data2)
	local_name.append(data2)

## 행정동 코드 천안시 데이터프레임과 합치기

In [9]:
cheonahn['행정동코드']=amd_code
cheonahn['법정동코드']=legal_code
cheonahn['지역명']=amd_name
cheonahn['짧은이름']=local_name
cheonahn

/var/folders/jv/jrhq9b3j1r71dg847_hrmn_m0000gn/T/ipykernel_73727/2006202373.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cheonahn['행정동코드']=amd_code
/var/folders/jv/jrhq9b3j1r71dg847_hrmn_m0000gn/T/ipykernel_73727/2006202373.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cheonahn['법정동코드']=legal_code
/var/folders/jv/jrhq9b3j1r71dg847_hrmn_m0000gn/T/ipykernel_73727/2006202373.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

,발생년,발생년월일시,주야,요일,사망자수,사상자수,중상자수,경상자수,부상신고자수,발생지시도,...,당사자종별_1당_대분류,당사자종별_2당_대분류,발생위치X_UTMK,발생위치Y_UTMK,경도,위도,행정동코드,법정동코드,지역명,짧은이름
38,2012,2012011212,주간,목,1,1,0,0,0,충남,...,화물차,화물차,966331,1857321,127.123003,36.713312,15131320,4413132026,천안시동남구 광덕면,광덕면
68,2012,2012010507,주간,목,1,1,0,0,0,충남,...,승용차,보행자,981851,1862244,127.296666,36.758114,15131360,4413136022,천안시동남구 병천면,병천면
187,2012,2012012800,야간,토,1,3,1,1,0,충남,...,승합차,승용차,983098,1858677,127.310718,36.725983,15131350,4413135026,천안시동남구 수신면,수신면
273,2012,2012010714,주간,토,3,7,2,2,0,충남,...,화물차,승용차,966259,1870721,127.121607,36.834104,15133103,4413310300,천안시서북구 백석동,백석동
274,2012,2012020506,주간,일,1,4,3,0,0,충남,...,승용차,승용차,979875,1859422,127.274604,36.732634,15131350,4413135021,천안시동남구 수신면,수신면
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39825,2021,2021092804,야간,화,1,1,0,0,0,충남,...,화물차,보행자,968920,1875488,127.151253,36.877167,15133256,4413325626,천안시서북구 직산읍,직산읍
39857,2021,2021101811,주간,월,1,1,0,0,0,충남,...,화물차,보행자,965814,1885110,127.115967,36.963796,15133250,4413325031,천안시서북구 성환읍,성환읍
39877,2021,2021110306,주간,수,1,1,0,0,0,충남,...,승용차,보행자,966061,1871958,127.119331,36.845248,15133590,4413310500,천안시서북구 부성2동,부성2동
39917,2021,2021120510,주간,일,1,1,0,0,0,충남,...,이륜차,승용차,965077,1868194,127.108468,36.811281,15133566,4413310800,천안시서북구 불당1동,불당1동


In [18]:
cleaned_data = [re.sub(r'\d+', '', item) for item in amd_name]

# Print the cleaned data
for item in cleaned_data:
    print(item)
print(len(cleaned_data))

천안시동남구 광덕면
천안시동남구 병천면
천안시동남구 수신면
천안시서북구 백석동
천안시동남구 수신면
천안시동남구 목천읍
천안시서북구 쌍용동
천안시동남구 청룡동
천안시동남구 일봉동
천안시서북구 입장면
천안시서북구 쌍용동
천안시서북구 성거읍
천안시서북구 성환읍
천안시서북구 성환읍
천안시동남구 신안동
천안시동남구 청룡동
천안시동남구 원성동
천안시동남구 병천면
천안시서북구 성정동
천안시동남구 광덕면
천안시동남구 동면
천안시동남구 일봉동
천안시동남구 광덕면
천안시서북구 직산읍
천안시동남구 문성동
천안시동남구 신안동
천안시동남구 신안동
천안시서북구 성정동
천안시서북구 입장면
천안시동남구 신안동
천안시서북구 백석동
천안시동남구 동면
천안시동남구 성남면
천안시서북구 쌍용동
천안시서북구 불당동
천안시서북구 성환읍
천안시서북구 성환읍
천안시서북구 입장면
천안시서북구 성정동
천안시동남구 병천면
천안시서북구 입장면
천안시서북구 성거읍
천안시동남구 신안동
천안시서북구 직산읍
천안시서북구 부성동
천안시동남구 목천읍
천안시동남구 수신면
천안시동남구 신안동
천안시동남구 동면
천안시동남구 광덕면
천안시서북구 쌍용동
천안시서북구 쌍용동
천안시서북구 성환읍
천안시서북구 쌍용동
천안시서북구 성정동
천안시서북구 백석동
천안시서북구 성환읍
천안시서북구 입장면
천안시서북구 입장면
천안시동남구 봉명동
천안시동남구 신안동
천안시동남구 광덕면
천안시동남구 신방동
천안시서북구 쌍용동
천안시동남구 신안동
천안시동남구 수신면
천안시서북구 성거읍
천안시동남구 청룡동
천안시동남구 목천읍
천안시동남구 성남면
천안시서북구 성환읍
천안시동남구 신방동
천안시동남구 신방동
천안시서북구 성환읍
천안시서북구 성환읍
천안시동남구 신안동
천안시서북구 부성동
천안시동남구 원성동
천안시동남구 봉명동
천안시서북구 입장면
천안시동남구 신방동
천안시동남구 성남면
천안시동남구 목천읍
천안시동남구 중앙동
천안시동남구 성남면
천안시동남구 신방동
천안시동남구 원성동
천안시동남구 신안동
천안시동남구 목천읍
천안시서북구 성거읍
천안시동남구 신안동
천안

In [11]:
cheonahn['읍면동']=cleaned_data

/var/folders/jv/jrhq9b3j1r71dg847_hrmn_m0000gn/T/ipykernel_73727/4054719175.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cheonahn['읍면동']=cleaned_data


## 좌표 추출 이후 이상치 제거

In [23]:
cheonahn = cheonahn[cheonahn['읍면동'].str.contains('천안시')]

,발생년,발생년월일시,주야,요일,사망자수,사상자수,중상자수,경상자수,부상신고자수,발생지시도,...,당사자종별_2당_대분류,발생위치X_UTMK,발생위치Y_UTMK,경도,위도,행정동코드,법정동코드,지역명,읍면동,짧은 이름
0,2012,2012011212,주간,목,1,1,0,0,0,충남,...,화물차,966331,1857321,127.123003,36.713312,15131320,4413132026,천안시동남구 광덕면,천안시동남구 광덕면,광덕면
1,2012,2012010507,주간,목,1,1,0,0,0,충남,...,보행자,981851,1862244,127.296666,36.758114,15131360,4413136022,천안시동남구 병천면,천안시동남구 병천면,병천면
2,2012,2012012800,야간,토,1,3,1,1,0,충남,...,승용차,983098,1858677,127.310718,36.725983,15131350,4413135026,천안시동남구 수신면,천안시동남구 수신면,수신면
3,2012,2012010714,주간,토,3,7,2,2,0,충남,...,승용차,966259,1870721,127.121607,36.834104,15133103,4413310300,천안시서북구 백석동,천안시서북구 백석동,백석동
4,2012,2012020506,주간,일,1,4,3,0,0,충남,...,승용차,979875,1859422,127.274604,36.732634,15131350,4413135021,천안시동남구 수신면,천안시동남구 수신면,수신면
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,2021,2021092804,야간,화,1,1,0,0,0,충남,...,보행자,968920,1875488,127.151253,36.877167,15133256,4413325626,천안시서북구 직산읍,천안시서북구 직산읍,직산읍
506,2021,2021101811,주간,월,1,1,0,0,0,충남,...,보행자,965814,1885110,127.115967,36.963796,15133250,4413325031,천안시서북구 성환읍,천안시서북구 성환읍,성환읍
507,2021,2021110306,주간,수,1,1,0,0,0,충남,...,보행자,966061,1871958,127.119331,36.845248,15133590,4413310500,천안시서북구 부성2동,천안시서북구 부성동,부성동
508,2021,2021120510,주간,일,1,1,0,0,0,충남,...,승용차,965077,1868194,127.108468,36.811281,15133566,4413310800,천안시서북구 불당1동,천안시서북구 불당동,불당동


In [20]:
cleaned_data = [re.sub(r'\d+', '', item) for item in local_name]

# Print the cleaned data
for item in cleaned_data:
    print(item)
cheonahn.drop('짧은이름',axis=1,inplace=True)
cheonahn['짧은 이름']=cleaned_data

광덕면
병천면
수신면
백석동
수신면
목천읍
쌍용동
청룡동
일봉동
입장면
쌍용동
성거읍
성환읍
성환읍
신안동
청룡동
원성동
병천면
성정동
광덕면
동면
일봉동
광덕면
직산읍
문성동
신안동
신안동
성정동
입장면
신안동
백석동
동면
성남면
쌍용동
불당동
성환읍
성환읍
입장면
성정동
병천면
입장면
성거읍
신안동
직산읍
부성동
목천읍
수신면
신안동
동면
광덕면
쌍용동
쌍용동
성환읍
쌍용동
성정동
백석동
성환읍
입장면
입장면
봉명동
신안동
광덕면
신방동
쌍용동
신안동
수신면
성거읍
청룡동
목천읍
성남면
성환읍
신방동
신방동
성환읍
성환읍
신안동
부성동
원성동
봉명동
입장면
신방동
성남면
목천읍
중앙동
성남면
신방동
원성동
신안동
목천읍
성거읍
신안동
불당동
성정동
불당동
성정동
부성동
불당동
부성동
성환읍
성정동
부성동
성환읍
부성동
쌍용동
직산읍
성정동
부성동
부성동
중앙동
성환읍
광덕면
입장면
일봉동
수신면
원성동
성정동
성남면
병천면
원성동
성남면
백석동
신방동
풍세면
신안동
성거읍
북면
쌍용동
풍세면
성정동
신안동
부성동
목천읍
청룡동
신안동
병천면
원성동
신방동
목천읍
부성동
성정동
직산읍
입장면
성환읍
일봉동
백석동
신방동
신안동
성정동
부성동
신방동
목천읍
원성동
신방동
청룡동
신안동
성남면
북면
광덕면
부성동
광덕면
성정동
쌍용동
원성동
동면
백곡면
입장면
성정동
성정동
입장면
부성동
입장면
불당동
성거읍
직산읍
성거읍
성정동
성거읍
신방동
동면
백석동
성거읍
수신면
쌍용동
풍세면
풍세면
목천읍
성거읍
부성동
불당동
미양면
부성동
쌍용동
풍세면
불당동
신방동
북면
목천읍
입장면
신방동
부성동
쌍용동
일봉동
쌍용동
풍세면
성환읍
광덕면
청룡동
백석동
목천읍
북면
목천읍
원성동
성환읍
입장면
신방동
성환읍
성환읍
신방동
직산읍
부성동
신안동
성거읍
직산읍
부성동
성환읍
성환읍
성정동
성환읍
성정동
성거읍
직산읍
입장면
백석동
입장면
부성동
부성동
목천읍
성환읍
성환읍
청룡동
문성동
성환읍
광덕면
목천읍
부성동
신방동
동면
수신면
청룡동
광덕면
목천읍
성환읍
성남

/var/folders/jv/jrhq9b3j1r71dg847_hrmn_m0000gn/T/ipykernel_73727/2432856314.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cheonahn.drop('짧은이름',axis=1,inplace=True)
/var/folders/jv/jrhq9b3j1r71dg847_hrmn_m0000gn/T/ipykernel_73727/2432856314.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cheonahn['짧은 이름']=cleaned_data


In [21]:
cheonahn.to_excel('CA.xlsx', index=False)  # Set index=False to exclude row numbers

# folium 으로 지도 시각화 해보기

### 지도 데이터 천안시만 추출하기

In [8]:
# json 파싱을 위한 패키지
import json


# 행정 구역 경계와 관련된 json 파일을 열어서 파싱
try:
	geo_data=json.load(open('./HangJeongDong_ver20230701.geojson',encoding='utf-8'))
except:
	geo_data=json.load(open('./HangJeongDong_ver20230701.geojson',encoding='utf-8-sig'))

# 천안시데이터만 추출. 
cheonan_features = [feature for feature in geo_data['features'] if '천안시' in feature['properties']['adm_nm']]
cheonan_json = {
    "type": "FeatureCollection",
    "features": cheonan_features
}

cheonan_json

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'adm_nm': '충청남도 천안시동남구 목천읍',
    'adm_cd': '3401111',
    'adm_cd2': '4413125000',
    'sgg': '44131',
    'sido': '44',
    'sidonm': '충청남도',
    'temp': '천안시동남구 목천읍',
    'sggnm': '천안시동남구',
    'adm_cd8': '34011110'},
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[127.25463712744555, 36.76159612945159],
       [127.25355407642138, 36.76081870300158],
       [127.2528017145003, 36.760071137628],
       [127.25258306711854, 36.759847161936875],
       [127.2522767062462, 36.759448061702074],
       [127.252171443022, 36.75924487311895],
       [127.2521269088359, 36.75895742638657],
       [127.25225629865237, 36.75853635801837],
       [127.25231618929854, 36.758192595253824],
       [127.25001878128774, 36.75661942655951],
       [127.24976565843772, 36.75650772610687],
       [127.24934712580308, 36.75645950951567],
       [127.24880164942908, 36.756424215752574],
       [127.2467343170

### 점으로 찍은 지도

In [26]:
m=folium.Map(location=[36.815129,127.23],zoom_start=11)

In [27]:
folium.Choropleth(geo_data=cheonan_json,fill_color='None',line_opacity=1,line_weight=2).add_to(m)

In [28]:
length=len(cheonahn)
for i in range(0,length):
    x=cheonahn.iloc[i]['경도']
    y=cheonahn.iloc[i]['위도']
    reason=cheonahn.iloc[i]['법규위반']
    locate=[y,x]
    dead=cheonahn.iloc[i]['사망자수']
    for _ in range(int(dead)):
        folium.CircleMarker(location=locate,radius=5,popup=reason,fill=True,color='None',fill_opacity=0.3,fill_color='blue').add_to(m)

In [29]:
folium.GeoJson(
    cheonan_json,
    name='geojson',
    style_function=lambda x: {'color': 'transparent', 'fillColor': 'transparent'},
    highlight_function=lambda x: {'weight': 0, 'fillOpacity': 0},
    tooltip=folium.GeoJsonTooltip(fields=['temp'], aliases=['지역명:'])
).add_to(m)

In [30]:
m

### 지도의 확대비율에 따른 군집화

In [17]:
m1=folium.Map(location=[36.815129,127.23],zoom_start=11)

In [18]:
folium.Choropleth(geo_data=cheonan_json,fill_color='None',line_opacity=1,line_weight=2).add_to(m1)

In [19]:
dong_clusters = {}

for i in range(len(cheonahn)):
    x = cheonahn.iloc[i]['경도']
    y = cheonahn.iloc[i]['위도']
    reason = cheonahn.iloc[i]['법규위반']
    dead = cheonahn.iloc[i]['사망자수']
    dong_code = cheonahn.iloc[i]['행정동코드']
    
    # Create a CircleMarker for each death
    for _ in range(int(dead)):
        if dong_code not in dong_clusters:
            dong_clusters[dong_code] = MarkerCluster().add_to(m1)
        folium.CircleMarker(location=[y, x], radius=5, popup=reason, fill=True, color='blue').add_to(dong_clusters[dong_code])

In [20]:
folium.GeoJson(
    cheonan_json,
    name='geojson',
    style_function=lambda x: {'color': 'transparent', 'fillColor': 'transparent'},
    highlight_function=lambda x: {'weight': 0, 'fillOpacity': 0},
    tooltip=folium.GeoJsonTooltip(fields=['temp'], aliases=['지역명:'])
).add_to(m1)

In [21]:
m1

### 법정동 별 계층화

In [34]:
result = cheonahn.groupby("지역명")["사망자수"].sum().reset_index()
result=result.drop([0,1])
result=result.reset_index(drop=True)
result

,지역명,사망자수
0,천안시동남구 광덕면,24
1,천안시동남구 동면,9
2,천안시동남구 목천읍,31
3,천안시동남구 문성동,4
4,천안시동남구 병천면,12
5,천안시동남구 봉명동,6
6,천안시동남구 북면,8
7,천안시동남구 성남면,17
8,천안시동남구 수신면,19
9,천안시동남구 신방동,20


## 법정동 으로 본 사고 계층화

In [23]:
g_map=folium.Map(location=[36.815129,127.23],zoom_start=11,tiles='Stamen Terrain')

In [24]:
folium.Choropleth(geo_data=cheonan_json,data=result,columns=('지역명','사망자수'),fill_color='YlOrRd',
                  fill_opacity=0.7,line_opacity=0.3,threshold_scale=[3,10,17,25,30,35,40,44],key_on='feature.properties.temp',).add_to(g_map)

In [25]:
# cheonan_features - 천안시 법정동 좌표경계
# 지도 생성

folium.GeoJson(
    cheonan_json,
    name='geojson',
    style_function=lambda x: {'color': 'transparent', 'fillColor': 'transparent'},
    highlight_function=lambda x: {'weight': 0, 'fillOpacity': 0},
    tooltip=folium.GeoJsonTooltip(fields=['temp'], aliases=['지역명:'])
).add_to(g_map)


g_map

In [22]:
df=pd.read_excel('./CA.xlsx')

In [17]:
df.to_excel('천안시 사망자 데이터.xlsx', index=False)  # Set index=False to exclude row numbers